In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = {i.split(".csv")[0] for i in os.listdir("./cases/overflow/high_carbs/")}
zinc_in_polymerase = set(pandas.read_csv("zinc_in_polymerase.txt",index_col=0,header=None).index.to_list())

In [4]:
run_for = ["Methylobacterium_populi_BJ001"]
len(run_for)

1

In [1]:
carbs = "EX_fru(e),EX_glc(e),EX_glc_D(e),EX_gal(e),EX_man(e),EX_mnl(e),EX_fuc_L(e),EX_glcn(e),EX_rmn(e),EX_arab_L(e),EX_drib(e),EX_rib_D(e),EX_xyl_D(e),EX_oxa(e),EX_lcts(e),EX_malt(e),EX_sucr(e),EX_melib(e),EX_cellb(e),EX_tre(e),EX_strch1(e)"
exchanges = carbs.split(",")

In [11]:
from diets import load_model,constrain_exchanges,get_bounds
from overflow import get_overflow
import coralme

conditions = ["high"]
metabolite = "carbs"

for c in conditions:
    name = "{}_{}".format(c,metabolite)
    fluxdir = "./cases/fluxes/{}".format(name)
    overflowdir = "./cases/overflow/{}".format(name)
    if not os.path.isdir(fluxdir):
        os.makedirs(fluxdir)
        os.makedirs(overflowdir)

def run(org):
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    sol0 = model.solution
    df0 = sol0.to_frame()
    # Exchanges
    rlist = exchanges
    # Run for all conditions
    for condition in conditions:
        name = "{}_{}".format(condition,metabolite)
        dct = {k:-2 for k in rlist if k in model.reactions}
        constrain_exchanges(model,dct)
        model.optimize()
        model.solution.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(name,org))
        get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(name,org))
        

In [15]:
# org = "Ruminococcus_torques_ATCC_27756"
# model = run(org)
# model.optimize(max_mu = 0.002916*2)
# model.solution.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(name,org))
# get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(name,org))

Read LP format model from file /tmp/tmpw3gkdnn8.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpn9po4k2x.lp
Reading time = 0.00 seconds
: 901 rows, 1988 columns, 8424 nonzeros
Read LP format model from file /tmp/tmp4ci359n7.lp
Reading time = 0.00 seconds
: 924 rows, 1986 columns, 8304 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.0029160000000000	Optimal
        2	0.0043740000000000	Not feasible
        3	0.0036450000000000	Not feasible
        4	0.0032805000000000	Not feasible
        5	0.0030982500000000	Not feasible
        6	0.0030071250000000	Not feasible
        7	0.0029615625000000	Not feasible
        8	0.0029387812500000	Not feasible
        9	0.0029273906250000	Not feasible
       10	0.0029216953125000	Not feasible
       11	0.0029188476562500	Not feasible
       12	0.0029174238281250	Not feasible
       13	0.0029167119140625	Not feasible


In [7]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (1 threads):   0%|                               | 0/1 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpktlm5l8t.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp67rd2sqz.lp
Reading time = 0.00 seconds
: 1343 rows, 3202 columns, 13882 nonzeros
Read LP format model from file /tmp/tmp12j4us20.lp
Reading time = 0.00 seconds
: 1355 rows, 3200 columns, 13712 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8	0.0109767817867390	Not feasible
        9	0.0054883908933695	Not feasible
       10	0.0027441954466847	Not feasible
       11	0.0013720977233424	Optimal
       12	0.0

Building (1 threads): 100%|██████████████████████| 1/1 [06:15<00:00, 375.47s/it]